In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.util import ngrams
import spacy
# from spacy import displacy # Descomenta si quieres usarlo para visualizar árboles

# --- 0. Configuración Inicial y Descarga de Recursos ---
# Descargar recursos de NLTK (solo la primera vez que se ejecute en un entorno nuevo)
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

# Descargar el modelo de español de SpaCy (solo la primera vez)
# Asegúrate de haber instalado spacy previamente: pip install spacy
# y luego, en tu terminal: python -m spacy download es_core_news_sm
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    print("Descargando modelo de español de SpaCy (es_core_news_sm)...")
    spacy.cli.download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")

# --- 1. Cargar tu Dataset ---
# --- ¡¡¡ CONFIGURA ESTAS DOS LÍNEAS SEGÚN TU ARCHIVO !!! ---
file_path = "data/reviews_filmaffinity_limpio.csv" # <--- ¡CAMBIA ESTO A LA RUTA Y NOMBRE DE TU ARCHIVO!
file_type = "csv" # <--- ¡CAMBIA ESTO A "csv" SI TU ARCHIVO ES CSV!

if file_type == "csv":
    df = pd.read_csv(file_path)
elif file_type == "excel":
    df = pd.read_excel(file_path)
else:
    raise ValueError("Tipo de archivo no soportado. Usa 'csv' o 'excel'.")

# --- ¡¡¡ VERIFICA ESTOS NOMBRES DE COLUMNA EN TU DATASET !!! ---
# Ajusta si tus columnas tienen nombres diferentes en tu archivo
GENRE_COLUMN = 'gender'
REVIEW_TEXT_COLUMN = 'review_text'

# Renombrar columnas para consistencia si no tienen los nombres esperados
if GENRE_COLUMN not in df.columns:
    print(f"ERROR: La columna de género '{GENRE_COLUMN}' no se encontró. Por favor, ajusta 'GENRE_COLUMN' o renombra tu columna en el archivo.")
    exit()
if REVIEW_TEXT_COLUMN not in df.columns:
    print(f"ERROR: La columna de texto de reseña '{REVIEW_TEXT_COLUMN}' no se encontró. Por favor, ajusta 'REVIEW_TEXT_COLUMN' o renombra tu columna en el archivo.")
    exit()

# Asegurarse de que los géneros sean limpios para la clasificación
# Por ejemplo, de 'Ocho apellido comedia' -> 'comedia'
# Esto asume que el género relevante es la última palabra de la cadena en 'film_genre'
df['genre'] = df[GENRE_COLUMN].apply(lambda x: str(x).split()[-1].lower()) # Asegura que x sea string

print("--- DataFrame Original (primeras 5 filas) ---")
print(df.head())
print(f"\nNúmero total de comentarios cargados: {len(df)}")
print("\n" + "="*50 + "\n")


# --- PARTE 2: LIMPIEZA DE DATOS (PREPROCESAMIENTO) ---
print("--- INICIANDO LIMPIEZA DE DATOS ---")

# 2.1 Función de Preprocesamiento Integral
lemmatizer = WordNetLemmatizer()
stop_words_spanish = set(stopwords.words('spanish'))

def clean_text_general(text):
    """
    Realiza una limpieza general del texto:
    - Convierte a minúsculas.
    - Decodifica caracteres especiales (tildes, eñes mal codificadas).
    - Elimina URLs, puntuación y números.
    """
    if not isinstance(text, str): # Asegura que la entrada sea string
        text = str(text)

    text = text.lower() # Convierte a minúsculas

    # Decodificación de caracteres comunes en español que pueden venir mal codificados
    text = text.replace('Ã¡', 'á').replace('Ã©', 'é').replace('Ã­', 'í').replace('Ã³', 'ó').replace('Ãº', 'ú')
    text = text.replace('Ã±', 'ñ').replace('â€™', "'").replace('â€œ', '"').replace('â€ ', '"')
    text = text.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u') # Normalización sin tildes para evitar problemas con algunos tokenizadores o modelos que no las manejen bien
    text = text.replace('ñ', 'n')


    # Eliminar URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Eliminar puntuación y caracteres especiales (solo deja letras y espacios)
    text = re.sub(r'[^a-záéíóúüñ\s]', '', text) # Deja letras (incluyendo acentuadas y ñ) y espacios

    # Eliminar números (ya los elimina la regex anterior, pero por si acaso)
    # text = re.sub(r'\d+', '', text)

    # Eliminar múltiples espacios en blanco
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def spacy_preprocess(text):
    """
    Procesa texto con SpaCy para obtener lemas limpios (sin stopwords y no alfanuméricos).
    """
    if not isinstance(text, str):
        text = str(text)
    
    doc = nlp(text)
    # is_alpha: filtra puntuación y números
    # is_stop: filtra stopwords
    lemmas = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
    return ' '.join(lemmas)

# Aplicar la limpieza general a la columna de comentarios
df['cleaned_review_general'] = df[REVIEW_TEXT_COLUMN].apply(clean_text_general)
print("\n--- Comentarios después de limpieza general (primeras 5 filas) ---")
print(df[[REVIEW_TEXT_COLUMN, 'cleaned_review_general']].head())


# Aplicar SpaCy para lematización y eliminación de stopwords
# Usaremos nlp.pipe para mayor eficiencia si el dataset es grande
print("\nProcesando comentarios con SpaCy para lematización y eliminación de stopwords...")
# Esto es más eficiente que .apply(lambda x: spacy_preprocess(x)) para datasets grandes
docs = nlp.pipe(df['cleaned_review_general'].astype(str), batch_size=500, n_process=-1) # n_process=-1 usa todos los núcleos disponibles
df['spacy_doc'] = list(docs) # Almacena los objetos Doc de SpaCy

# Extraer lemas limpios de los objetos Doc
df['cleaned_review_spacy_lemmas'] = df['spacy_doc'].apply(
    lambda doc: ' '.join([token.lemma_ for token in doc if token.is_alpha and not token.is_stop])
)

print("\n--- Comentarios después de SpaCy (lemas limpios) (primeras 5 filas) ---")
print(df[[REVIEW_TEXT_COLUMN, 'cleaned_review_spacy_lemmas']].head())
print("\n" + "="*50 + "\n")


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Angelica\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


--- DataFrame Original (primeras 5 filas) ---
               film_name   gender  film_avg_rate  review_rate  \
0  Ocho apellidos vascos  Comedia            6.0          3.0   
1  Ocho apellidos vascos  Comedia            6.0          2.0   
2  Ocho apellidos vascos  Comedia            6.0          2.0   
3  Ocho apellidos vascos  Comedia            6.0          2.0   
4  Ocho apellidos vascos  Comedia            6.0          2.0   

                                        review_title  \
0     OCHO APELLIDOS VASCOS...Y NINGÚN NOMBRE PROPIO   
1                                     El perro verde   
2  Si no eres de comer mierda... no te comas esta...   
3                                    Aida: The movie   
4               UN HOMBRE SOLO (Julio Iglesias 1987)   

                                         review_text    genre  
0  La mayor virtud de esta película es su existen...  comedia  
1  No soy un experto cinéfilo, pero pocas veces m...  comedia  
2  Si no eres un incondicional del

In [ ]:
# --- PARTE 3: ESTADÍSTICA A PARTIR DE LOS COMENTARIOS ---
print("--- INICIANDO ANÁLISIS ESTADÍSTICO ---")

# --- 3.1. Conteo y Estadísticas Básicas ---

# Número de palabras por comentario (después de limpieza)
df['num_palabras_limpias'] = df['cleaned_review_spacy_lemmas'].apply(lambda x: len(x.split()))

# Número de oraciones por comentario (antes de la limpieza profunda, para mantener la estructura original)
df['num_oraciones'] = df[REVIEW_TEXT_COLUMN].apply(
    lambda x: len(nltk.sent_tokenize(str(x), language='spanish')) if pd.notna(x) else 0
)

print("\n--- Estadísticas de longitud de comentarios (primeras 5 filas) ---")
print(df[[REVIEW_TEXT_COLUMN, 'num_palabras_limpias', 'num_oraciones']].head())

# Estadísticas descriptivas de longitud por género
print("\n--- Estadísticas descriptivas de Número de Palabras Limpias por Género ---")
print(df.groupby('genre')['num_palabras_limpias'].describe())

print("\n--- Estadísticas descriptivas de Número de Oraciones por Género ---")
print(df.groupby('genre')['num_oraciones'].describe())


# --- 3.2. Frecuencia de Palabras y N-gramas por Género ---

def get_top_n_items(corpus, n_items=None, item_type='word', n_gram_size=1):
    """
    Obtiene los N ítems más frecuentes (palabras o n-gramas) de un corpus.
    item_type: 'word' para palabras, 'ngram' para n-gramas.
    n_gram_size: tamaño del n-grama si item_type es 'ngram'.
    """
    vec = Counter()
    for review in corpus:
        tokens = str(review).split() # Asegura que review sea string
        if item_type == 'word':
            for word in tokens:
                vec[word] += 1
        elif item_type == 'ngram':
            if len(tokens) >= n_gram_size: # Asegura que hay suficientes tokens para el n-grama
                for ngram_item in ngrams(tokens, n_gram_size):
                    vec[' '.join(ngram_item)] += 1
    return vec.most_common(n_items)

print("\n--- Frecuencia de Palabras y N-gramas por Género ---")
all_genres = df['genre'].unique()

for genre_val in all_genres:
    genre_corpus = df[df['genre'] == genre_val]['cleaned_review_spacy_lemmas']

    print(f"\n***** Análisis para el género: '{genre_val.upper()}' *****")

    # Palabras más frecuentes
    top_words = get_top_n_items(genre_corpus, n_items=10, item_type='word')
    print(f"  Top 10 palabras (lemas): {top_words}")

    # Bigramas más frecuentes
    top_bigrams = get_top_n_items(genre_corpus, n_items=5, item_type='ngram', n_gram_size=2)
    print(f"  Top 5 bigramas (lemas): {top_bigrams}")

    # Trigramas más frecuentes
    top_trigrams = get_top_n_items(genre_corpus, n_items=3, item_type='ngram', n_gram_size=3)
    print(f"  Top 3 trigramas (lemas): {top_trigrams}")


# --- 3.3. Análisis con Expresiones Regulares ---

# Define patrones específicos para cada género. Estos son solo ejemplos, puedes expandirlos.
genre_patterns = {
    'comedia': [
        r'pelicula\s+(?:muy\s+)?(?:divertida|graciosa|entretenida|absurda)', # Usamos ?: para grupos no capturadores
        r'(?:reir|carcajad|humor\s+(?:excelente|buen|genial))'
    ],
    'drama': [
        r'historia\s+(?:profunda|emotiva|triste|reflexiva|real)',
        r'(?:llorar|conmovedor|impacto\s+emocional)'
    ],
    'thriller': [
        r'(?:intenso|suspenso|giro\s+inesperado|misterio|intriga)'
    ],
    'ciencia ficción': [ # Ajustado a 'ciencia ficcion' por la normalización sin tildes
        r'(?:efecto\s+especial|ciencia\s+ficcion|futurista|espacio|tecnologia)'
    ]
}

def apply_regex_patterns(text, patterns):
    """
    Aplica una lista de patrones regex a un texto y cuenta cuántos coinciden.
    Devuelve un contador de los géneros detectados.
    """
    if not isinstance(text, str):
        return [("N/A", 0)] # Devolver un valor por defecto si no es string

    detected_genres = Counter()
    for genre, regex_list in patterns.items():
        for pattern in regex_list:
            if re.search(pattern, text, re.IGNORECASE): # re.IGNORECASE para coincidir independientemente de mayúsculas/minúsculas
                detected_genres[genre] += 1
    
    if not detected_genres: # Si no se detectó ningún género, devuelve 'desconocido'
        return [("desconocido", 0)]
    return detected_genres.most_common(1) # Devuelve el género más detectado por esta técnica

df['predicted_genre_regex'] = df['cleaned_review_general'].apply(
    lambda x: apply_regex_patterns(x, genre_patterns)
)

print("\n--- Predicciones de Género Basadas en Expresiones Regulares (primeras 5 filas) ---")
for idx, row in df.head().iterrows():
    print(f"Comentario: '{row[REVIEW_TEXT_COLUMN]}'")
    print(f"Género Real: {row['genre']}")
    print(f"Género Predicho por Regex: {row['predicted_genre_regex']}")
    print("-" * 30)


# --- 3.4. Gramática Libre de Contexto (GLC) y Análisis Sintáctico de Dependencias con SpaCy ---

# Definir palabras clave para las reglas sintácticas (lemas, sin tildes por normalización)
genre_keywords_lemmas = {
    "comedia": ["divertir", "gracioso", "entretener", "absurdo", "ingenioso", "reir", "humor", "carcajad"],
    "drama": ["profundo", "emotivo", "triste", "reflexivo", "real", "conmovedor", "llorar", "impactar"],
    "thriller": ["intenso", "suspenso", "giro", "inesperado", "misterio", "intriga"],
    "ciencia ficcion": ["efecto", "especial", "ciencia", "ficcion", "futurista", "espacio", "tecnologia"]
}

def analyze_genre_patterns_spacy(doc, genre_keywords_lemmas):
    """
    Analiza un documento SpaCy para detectar patrones sintácticos que sugieran un género.
    """
    if not isinstance(doc, spacy.tokens.doc.Doc): # Manejar posibles valores no-Doc (ej. NaNs)
        return [("N/A", 0)]

    detected_genres = Counter()

    for sentence in doc.sents: # Iterar por cada oración del documento
        for token in sentence:
            # Regla 1: Adjetivo que describe 'película' o 'historia'
            if token.pos_ == "ADJ":
                # Si es un atributo de 'ser' (ej. "película es [adjetivo]")
                if token.head.lemma_ == "ser": # y el token modifica al verbo "ser"
                    for child_of_head in token.head.children:
                        # nsubj: nominal subject
                        if child_of_head.dep_ == "nsubj" and child_of_head.lemma_ in ["pelicula", "historia", "trama", "film"]:
                            for genre, keywords in genre_keywords_lemmas.items():
                                if token.lemma_ in keywords:
                                    detected_genres[genre] += 1

                # Si el adjetivo modifica directamente un sustantivo (ej. "divertida película")
                if token.dep_ == "amod" and token.head.lemma_ in ["pelicula", "historia", "trama", "film"]:
                    for genre, keywords in genre_keywords_lemmas.items():
                        if token.lemma_ in keywords:
                            detected_genres[genre] += 1

            # Regla 2: Verbos de sentimiento/reacción
            if token.pos_ == "VERB":
                for genre, keywords in genre_keywords_lemmas.items():
                    if token.lemma_ in keywords: # ej. "reír" para comedia, "llorar" para drama
                        detected_genres[genre] += 1

            # Regla 3: Sustantivos clave (ej. 'humor', 'suspenso', 'drama')
            if token.pos_ == "NOUN":
                for genre, keywords in genre_keywords_lemmas.items():
                    if token.lemma_ in keywords:
                        detected_genres[genre] += 0.5 # Menor peso, ya que pueden ser más ambiguos

            # Regla 4: Combinaciones de sustantivo + adjetivo clave (sin depender de un verbo central como 'ser')
            # Ej: 'humor absurdo' -> comedia, 'drama familiar' -> drama
            if token.pos_ == "ADJ" and token.head.pos_ == "NOUN": # adjetivo modificando un sustantivo
                if token.head.lemma_ == "humor" and token.lemma_ in ["absurdo", "ingenioso"]:
                    detected_genres["comedia"] += 1
                if token.head.lemma_ == "drama" and token.lemma_ in ["familiar", "historico", "psicologico"]:
                    detected_genres["drama"] += 1
                if token.head.lemma_ == "efecto" and token.lemma_ == "especial":
                    detected_genres["ciencia ficcion"] += 1
                if token.head.lemma_ == "giro" and token.lemma_ == "inesperado":
                    detected_genres["thriller"] += 1

    if not detected_genres:
        return [("desconocido", 0)]
    return detected_genres.most_common(1) # Devuelve el género más detectado

df['predicted_genre_spacy_rules'] = df['spacy_doc'].apply(
    lambda doc: analyze_genre_patterns_spacy(doc, genre_keywords_lemmas)
)

print("\n--- Predicciones de Género Basadas en Reglas Sintácticas (SpaCy) (primeras 5 filas) ---")
for idx, row in df.head().iterrows():
    print(f"Comentario: '{row[REVIEW_TEXT_COLUMN]}'")
    print(f"Género Real: {row['genre']}")
    print(f"Género Predicho por Reglas SpaCy: {row['predicted_genre_spacy_rules']}")
    print("-" * 30)

# Opcional: Visualizar un árbol de dependencias para entender mejor (solo si tienes la UI adecuada)
# from spacy import displacy
# print("\n--- Visualización de un Árbol de Dependencias (ejemplo) ---")
# # Asegúrate de que df['spacy_doc'][0] sea un objeto Doc válido (no NaN)
# if isinstance(df['spacy_doc'][0], spacy.tokens.doc.Doc):
#     displacy.render(df['spacy_doc'][0], style="dep", jupyter=True, options={"distance": 90})
# else:
#     print("El primer documento de SpaCy no es válido para visualización.")

print("\n--- Proceso Completado ---")
# Puedes guardar el DataFrame con las nuevas columnas si lo deseas
# df.to_excel("reviews_processed.xlsx", index=False)
# df.to_csv("reviews_processed.csv", index=False)

- ✔ Limpieza de texto (sin eliminar stopwords)
- ✔ Creación de columnas limpias
- ✔ Estadística descriptiva por género
- ✔ Análisis con expresiones regulares
- ✔ Análisis sintáctico + reglas de GLC con SpaCy

🧩 CÓDIGO FINAL COMPLETO — Limpieza + Estadística Sintáctica + GLC

In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from collections import Counter
import spacy

# --- PARTE 0: CONFIGURACIÓN INICIAL ---
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

# Descargar el modelo SpaCy español si no está disponible
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    import spacy.cli
    spacy.cli.download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")

# --- PARTE 1: CARGA DEL DATASET ---
file_path = "data/reviews_filmaffinity_limpio.csv"
df = pd.read_csv(file_path)

GENRE_COLUMN = 'gender'
REVIEW_TEXT_COLUMN = 'review_text'

df['genre'] = df[GENRE_COLUMN].apply(lambda x: str(x).split()[-1].lower())

print("--- DataFrame Original (primeras 5 filas) ---")
print(df.head(), "\n")

# --- PARTE 2: LIMPIEZA DE TEXTO (sin eliminar stopwords) ---
lemmatizer = WordNetLemmatizer()

def clean_text_general(text):
    if not isinstance(text, str):
        text = str(text)

    text = text.lower()
    text = text.replace('Ã¡','á').replace('Ã©','é').replace('Ã­','í').replace('Ã³','ó').replace('Ãº','ú')
    text = text.replace('Ã±','ñ').replace('â€™',"'").replace('â€œ','"').replace('â€','"')
    text = text.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u').replace('ñ','n')

    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'[^a-záéíóúüñ\s.,!?]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def spacy_preprocess(text):
    if not isinstance(text, str):
        text = str(text)
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc if token.is_alpha or token.text in [",", ".", "!", "?"]]
    return ' '.join(lemmas)

# Aplicar limpieza
df['cleaned_review_general'] = df[REVIEW_TEXT_COLUMN].apply(clean_text_general)

print("\n--- Ejemplo de limpieza general ---")
print(df[['review_text', 'cleaned_review_general']].head(), "\n")

# Procesar con SpaCy
docs = nlp.pipe(df['cleaned_review_general'].astype(str), batch_size=500, n_process=-1)
df['spacy_doc'] = list(docs)

df['cleaned_review_spacy_lemmas'] = df['spacy_doc'].apply(
    lambda doc: ' '.join([token.lemma_ for token in doc if token.is_alpha or token.text in [",", ".", "!", "?"]])
)

print("\n--- Ejemplo de texto lematizado ---")
print(df[['review_text', 'cleaned_review_spacy_lemmas']].head(), "\n")

# --- PARTE 3: ESTADÍSTICA SINTÁCTICA + GLC ---
print("--- INICIANDO ANÁLISIS ESTADÍSTICO ---")

# 3.1 Conteo y estadísticas básicas
df['num_palabras_limpias'] = df['cleaned_review_spacy_lemmas'].apply(lambda x: len(str(x).split()))
df['num_oraciones'] = df[REVIEW_TEXT_COLUMN].apply(
    lambda x: len(nltk.sent_tokenize(str(x), language='spanish')) if pd.notna(x) else 0
)

print("\n--- Estadísticas de longitud ---")
print(df[[REVIEW_TEXT_COLUMN, 'num_palabras_limpias', 'num_oraciones']].head(), "\n")

print("\n--- Estadísticas descriptivas por género ---")
print(df.groupby('genre')['num_palabras_limpias'].describe(), "\n")
print(df.groupby('genre')['num_oraciones'].describe(), "\n")

# 3.2 Frecuencia de palabras y n-gramas
def get_top_n_items(corpus, n_items=None, item_type='word', n_gram_size=1):
    vec = Counter()
    for review in corpus:
        tokens = str(review).split()
        if item_type == 'word':
            for word in tokens:
                vec[word] += 1
        elif item_type == 'ngram':
            if len(tokens) >= n_gram_size:
                for ngram_item in ngrams(tokens, n_gram_size):
                    vec[' '.join(ngram_item)] += 1
    return vec.most_common(n_items)

print("\n--- Frecuencia de Palabras y N-gramas ---")
for genre_val in df['genre'].unique():
    genre_corpus = df[df['genre'] == genre_val]['cleaned_review_spacy_lemmas']
    print(f"\n***** Género: {genre_val.upper()} *****")
    print("Top 10 palabras:", get_top_n_items(genre_corpus, 10))
    print("Top 5 bigramas:", get_top_n_items(genre_corpus, 5, 'ngram', 2))
    print("Top 3 trigramas:", get_top_n_items(genre_corpus, 3, 'ngram', 3))

# 3.3 Análisis con expresiones regulares
genre_patterns = {
    'comedia': [
        r'pelicula\s+(?:muy\s+)?(?:divertida|graciosa|entretenida|absurda)',
        r'(?:reir|carcajad|humor\s+(?:excelente|buen|genial))'
    ],
    'drama': [
        r'historia\s+(?:profunda|emotiva|triste|reflexiva|real)',
        r'(?:llorar|conmovedor|impacto\s+emocional)'
    ],
    'thriller': [r'(?:intenso|suspenso|giro\s+inesperado|misterio|intriga)'],
    'ciencia ficcion': [r'(?:efecto\s+especial|ciencia\s+ficcion|futurista|espacio|tecnologia)']
}

def apply_regex_patterns(text, patterns):
    if not isinstance(text, str):
        return [("N/A", 0)]
    detected_genres = Counter()
    for genre, regex_list in patterns.items():
        for pattern in regex_list:
            if re.search(pattern, text, re.IGNORECASE):
                detected_genres[genre] += 1
    if not detected_genres:
        return [("desconocido", 0)]
    return detected_genres.most_common(1)

df['predicted_genre_regex'] = df['cleaned_review_general'].apply(lambda x: apply_regex_patterns(x, genre_patterns))

print("\n--- Predicciones Regex (primeras 5 filas) ---")
for idx, row in df.head().iterrows():
    print(f"Comentario: '{row[REVIEW_TEXT_COLUMN]}'")
    print(f"Género real: {row['genre']}")
    print(f"Género regex: {row['predicted_genre_regex']}")
    print("-" * 30)

# 3.4 Gramática libre de contexto (GLC)
genre_keywords_lemmas = {
    "comedia": ["divertir", "gracioso", "entretener", "absurdo", "ingenioso", "reir", "humor", "carcajad"],
    "drama": ["profundo", "emotivo", "triste", "reflexivo", "real", "conmovedor", "llorar", "impactar"],
    "thriller": ["intenso", "suspenso", "giro", "inesperado", "misterio", "intriga"],
    "ciencia ficcion": ["efecto", "especial", "ciencia", "ficcion", "futurista", "espacio", "tecnologia"]
}

def analyze_genre_patterns_spacy(doc, genre_keywords_lemmas):
    if not isinstance(doc, spacy.tokens.doc.Doc):
        return [("N/A", 0)]

    detected_genres = Counter()

    for sentence in doc.sents:
        for token in sentence:
            if token.pos_ == "ADJ":
                if token.head.lemma_ == "ser":
                    for child in token.head.children:
                        if child.dep_ == "nsubj" and child.lemma_ in ["pelicula", "historia", "trama", "film"]:
                            for genre, keywords in genre_keywords_lemmas.items():
                                if token.lemma_ in keywords:
                                    detected_genres[genre] += 1
                if token.dep_ == "amod" and token.head.lemma_ in ["pelicula", "historia", "trama", "film"]:
                    for genre, keywords in genre_keywords_lemmas.items():
                        if token.lemma_ in keywords:
                            detected_genres[genre] += 1

            if token.pos_ == "VERB":
                for genre, keywords in genre_keywords_lemmas.items():
                    if token.lemma_ in keywords:
                        detected_genres[genre] += 1

            if token.pos_ == "NOUN":
                for genre, keywords in genre_keywords_lemmas.items():
                    if token.lemma_ in keywords:
                        detected_genres[genre] += 0.5

            if token.pos_ == "ADJ" and token.head.pos_ == "NOUN":
                if token.head.lemma_ == "humor" and token.lemma_ in ["absurdo", "ingenioso"]:
                    detected_genres["comedia"] += 1
                if token.head.lemma_ == "drama" and token.lemma_ in ["familiar", "historico", "psicologico"]:
                    detected_genres["drama"] += 1
                if token.head.lemma_ == "efecto" and token.lemma_ == "especial":
                    detected_genres["ciencia ficcion"] += 1
                if token.head.lemma_ == "giro" and token.lemma_ == "inesperado":
                    detected_genres["thriller"] += 1

    if not detected_genres:
        return [("desconocido", 0)]
    return detected_genres.most_common(1)

df['predicted_genre_spacy_rules'] = df['spacy_doc'].apply(lambda doc: analyze_genre_patterns_spacy(doc, genre_keywords_lemmas))

print("\n--- Predicciones SpaCy GLC (primeras 5 filas) ---")
for idx, row in df.head().iterrows():
    print(f"Comentario: '{row[REVIEW_TEXT_COLUMN]}'")
    print(f"Género real: {row['genre']}")
    print(f"Género SpaCy: {row['predicted_genre_spacy_rules']}")
    print("-" * 30)

print("\n--- PROCESO COMPLETADO ---")


c:\Users\Angelica\AppData\Local\Programs\Python\Python312\Lib\site-packages\nltk\metrics\association.py:26: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.3)
  from scipy.stats import fisher_exact
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Angelica\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


--- DataFrame Original (primeras 5 filas) ---
               film_name   gender  film_avg_rate  review_rate  \
0  Ocho apellidos vascos  Comedia            6.0          3.0   
1  Ocho apellidos vascos  Comedia            6.0          2.0   
2  Ocho apellidos vascos  Comedia            6.0          2.0   
3  Ocho apellidos vascos  Comedia            6.0          2.0   
4  Ocho apellidos vascos  Comedia            6.0          2.0   

                                        review_title  \
0     OCHO APELLIDOS VASCOS...Y NINGÚN NOMBRE PROPIO   
1                                     El perro verde   
2  Si no eres de comer mierda... no te comas esta...   
3                                    Aida: The movie   
4               UN HOMBRE SOLO (Julio Iglesias 1987)   

                                         review_text    genre  
0  La mayor virtud de esta película es su existen...  comedia  
1  No soy un experto cinéfilo, pero pocas veces m...  comedia  
2  Si no eres un incondicional del

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/spanish/[0m

  Searched in:
    - 'C:\\Users\\Angelica/nltk_data'
    - 'c:\\Users\\Angelica\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\Angelica\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\Angelica\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\Angelica\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
